In [1]:
import pandas as pd
import numpy as np
import pyodbc
from pathlib import Path

import matplotlib.pyplot as plt

import numpy as np
from scipy.interpolate import interp1d

In [2]:
# Configuración de los parámetros de conexión
server = 'localhost\\SQLEXPRESS'  # O el punto '.' que usaste en SSMS
database = 'EM_CUN'

# Cadena de conexión usando Autenticación de Windows (Trusted_Connection)
conn_str = (
    f'DRIVER={{ODBC Driver 17 for SQL Server}};'
    f'SERVER={server};'
    f'DATABASE={database};'
    f'Trusted_Connection=yes;'
)


In [3]:
query = "select * from dbo.estaciones_full;"
conn = pyodbc.connect(conn_str)
raw_data = pd.read_sql(query, conn)

conn.close()

C:\Users\User\AppData\Local\Temp\ipykernel_24936\341712462.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  raw_data = pd.read_sql(query, conn)


In [4]:
query = "select * from dbo.coordenadas_estaciones;"
conn = pyodbc.connect(conn_str)
coor = pd.read_sql(query, conn)
coor.drop(["tx_id"], axis=1, inplace=True)

conn.close()

C:\Users\User\AppData\Local\Temp\ipykernel_24936\2772575436.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  coor = pd.read_sql(query, conn)


In [5]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97652 entries, 0 to 97651
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   tx_id              97652 non-null  int64         
 1   codigo_estacion    97652 non-null  object        
 2   fecha_observacion  97652 non-null  datetime64[ns]
 3   sensor_pre         97652 non-null  object        
 4   sensor_vel         97652 non-null  object        
 5   sensor_dir         97652 non-null  object        
 6   sensor_tem         97652 non-null  object        
 7   presion            97652 non-null  float64       
 8   velocidad          97652 non-null  float64       
 9   direccion          97652 non-null  float64       
 10  temperatura        97652 non-null  float64       
dtypes: datetime64[ns](1), float64(4), int64(1), object(5)
memory usage: 8.2+ MB


# explo

## coordenadas

In [6]:
coor.apply(lambda df: f"{df["codigo_estacion"]}: ( {df["latitud"]} , {df["longitud"]} )", axis=1)

0     0035025060: ( 4.587 , -73.996 )
1     0021195120: ( 4.393 , -74.393 )
2     3502500135: ( 4.226 , -73.815 )
3     0023065507: ( 5.124 , -74.544 )
4     0021195190: ( 4.307 , -74.308 )
5     0024015110: ( 5.304 , -73.852 )
6     0021235030: ( 4.307 , -74.807 )
7     0021205700: ( 4.878 , -73.864 )
8     0023065190: ( 5.137 , -74.481 )
9     0023065180: ( 5.017 , -74.471 )
10     0021205970: ( 5.09 , -73.881 )
11    2306500106: ( 5.002 , -74.537 )
12     0021206280: ( 4.648 , -74.32 )
13    0035035130: ( 4.714 , -73.803 )
14    0021205670: ( 4.768 , -74.434 )
15    0023065120: ( 5.138 , -74.137 )
16     3505500121: ( 4.517 , -73.44 )
17    0023035030: ( 5.494 , -74.658 )
18     0021205517: ( 4.788 , -74.05 )
19    0021205513: ( 4.505 , -74.184 )
20    2306500204: ( 4.877 , -74.288 )
21    0021205526: ( 4.691 , -74.389 )
22    0035020300: ( 4.254 , -74.003 )
23     0021237060: ( 4.633 , -74.65 )
24    0021205940: ( 4.832 , -74.381 )
25    0021205420: ( 4.689 , -74.206 )
26    003506

## registro fechas

In [7]:
df = raw_data.copy()
df.drop("tx_id", axis=1, inplace=True)

In [8]:
print(df.shape)
print("-"*50)
cols = ["sensor_pre", "sensor_vel", "sensor_dir", "sensor_tem"]
for col in cols:
    print(df[col].value_counts())
    col_0 = df[col].value_counts().index[0]
    df = df[df[col] == col_0].copy()
    print("-"*50)
print(df.shape)

(97652, 10)
--------------------------------------------------
sensor_pre
0255    97652
Name: count, dtype: int64
--------------------------------------------------
sensor_vel
0103    97652
Name: count, dtype: int64
--------------------------------------------------
sensor_dir
0104    97652
Name: count, dtype: int64
--------------------------------------------------
sensor_tem
0068    97652
Name: count, dtype: int64
--------------------------------------------------
(97652, 10)


In [9]:
estaciones = df["codigo_estacion"].unique()

In [10]:
# df = df.sort_values(["codigo_estacion", "fecha_observacion"]).reset_index(drop=True)
df = df.sort_values(["fecha_observacion", "codigo_estacion"]).reset_index(drop=True)

In [11]:
df["fecha_observacion"].diff().unique()

<TimedeltaArray>
[NaT, '0 days 00:00:00', '0 days 00:10:00', '0 days 01:00:00',
 '0 days 00:40:00']
Length: 5, dtype: timedelta64[ns]

In [12]:
fecha_minima = pd.to_datetime("2025-12-15")
df = df[df["fecha_observacion"] >= fecha_minima]

In [13]:
estaciones_ok = list()
n = 1
for estacion in estaciones:
    min_date = df.loc[df["codigo_estacion"] == estacion, "fecha_observacion"].min()
    max_date = df.loc[df["codigo_estacion"] == estacion, "fecha_observacion"].max()
    diff_temp = df.loc[df["codigo_estacion"] == estacion, "fecha_observacion"].diff().unique()
    rows_temp = df.loc[df["codigo_estacion"] == estacion].shape[0]
    # if len(diff_temp) == 2: 
    # if rows_temp == 744 and not estacion in ["0024035250", "0024035360"]:
    # if rows_temp == 744:
    # if rows_temp == 408:
    # if min_date <= pd.to_datetime("2025-12-14") and len(diff_temp) == 2:
    # if rows_temp == 408 and not estacion in ["0024035250"]:
    if True:
        print(n , estacion)
        estaciones_ok.append(estacion)
        for diff in diff_temp: print(">", diff)
        print("cantidad de registros: ", rows_temp)
        print("fecha min: ", min_date)
        print("fecha max: ", max_date)
        print("-"*50)
        n += 1

1 0021205518
> NaT
> 0 days 00:00:00
> 0 days 01:00:00
> 0 days 00:10:00
> 0 days 00:30:00
> 0 days 00:50:00
> 0 days 02:00:00
> 0 days 03:00:00
cantidad de registros:  914
fecha min:  2025-12-15 00:00:00
fecha max:  2025-12-31 23:00:00
--------------------------------------------------
2 0021205521
> NaT
> 0 days 00:00:00
> 0 days 01:00:00
> 0 days 02:00:00
> 0 days 00:10:00
cantidad de registros:  830
fecha min:  2025-12-15 00:00:00
fecha max:  2025-12-31 23:00:00
--------------------------------------------------
3 0021205524
> NaT
> 0 days 00:00:00
> 0 days 00:10:00
> 0 days 00:20:00
> 0 days 00:40:00
> 0 days 01:00:00
> 0 days 00:30:00
> 0 days 01:10:00
cantidad de registros:  3304
fecha min:  2025-12-15 00:00:00
fecha max:  2025-12-26 21:00:00
--------------------------------------------------
4 0035025080
> NaT
> 0 days 00:00:00
> 0 days 01:00:00
> 0 days 02:00:00
> 0 days 03:00:00
> 0 days 05:00:00
> 0 days 04:00:00
cantidad de registros:  768
fecha min:  2025-12-15 00:00:00
fe

In [14]:
coor[coor["codigo_estacion"].isin(estaciones_ok)].apply(lambda df: f"{df["codigo_estacion"]}: ( {df["latitud"]} , {df["longitud"]} )", axis=1)

0     0035025060: ( 4.587 , -73.996 )
1     0021195120: ( 4.393 , -74.393 )
4     0021195190: ( 4.307 , -74.308 )
5     0024015110: ( 5.304 , -73.852 )
6     0021235030: ( 4.307 , -74.807 )
7     0021205700: ( 4.878 , -73.864 )
9     0023065180: ( 5.017 , -74.471 )
11    2306500106: ( 5.002 , -74.537 )
12     0021206280: ( 4.648 , -74.32 )
13    0035035130: ( 4.714 , -73.803 )
14    0021205670: ( 4.768 , -74.434 )
15    0023065120: ( 5.138 , -74.137 )
17    0023035030: ( 5.494 , -74.658 )
18     0021205517: ( 4.788 , -74.05 )
19    0021205513: ( 4.505 , -74.184 )
20    2306500204: ( 4.877 , -74.288 )
21    0021205526: ( 4.691 , -74.389 )
22    0035020300: ( 4.254 , -74.003 )
23     0021237060: ( 4.633 , -74.65 )
24    0021205940: ( 4.832 , -74.381 )
25    0021205420: ( 4.689 , -74.206 )
26    0035060210: ( 4.649 , -73.641 )
27    2120500050: ( 5.159 , -74.023 )
28    0021206950: ( 5.235 , -73.527 )
29    0024015508: ( 4.308 , -74.282 )
30    0021205525: ( 5.229 , -73.585 )
31    002119

In [15]:
print(f"rows: {df.shape[0]:,}")
wsdata = df.loc[df["codigo_estacion"].isin(estaciones_ok)].copy()
wsdata.drop(["sensor_pre", "sensor_vel", "sensor_dir", "sensor_tem"], axis=1, inplace=True)
print(f"rows: {wsdata.shape[0]:,}")
print(f"{100 * wsdata.shape[0] / df.shape[0]:,.2f} %")

rows: 52,868
rows: 52,868
100.00 %


# WSDATA

In [16]:
wsdata.head()

,codigo_estacion,fecha_observacion,presion,velocidad,direccion,temperatura
44784,0021195120,2025-12-15,856.3,2.1,11.0,20.3
44785,0021195120,2025-12-15,856.3,2.1,11.0,20.3
44786,0021195170,2025-12-15,719.3,2.3,125.0,13.5
44787,0021195170,2025-12-15,719.3,2.3,125.0,13.5
44788,0021195190,2025-12-15,784.9,2.5,114.0,14.0


In [17]:
wsdata = wsdata.sort_values(["codigo_estacion", "fecha_observacion"]).reset_index(drop=True)

In [18]:
wsdata["codigo_estacion"].value_counts()

codigo_estacion
0021205519    4886
0021205515    4886
0021205526    4862
0021205517    4586
0021237060    3930
0021205524    3304
0021205518     914
0021205513     876
0021205521     830
0021195170     816
0021235030     816
2120500050     816
0021195120     816
0021205660     816
0021205940     816
0023065120     816
0021206280     816
0021205670     816
2120500135     816
0024015380     816
2401500040     816
2306500106     816
0024015110     816
0023125170     816
0035060210     810
0035020300     806
2306500204     806
3505500061     796
0021205700     792
0023035030     790
0021205525     790
0021206950     780
0024015508     772
0035025080     768
0035035130     754
0023065110     716
0021195190     712
0023065180     652
0021205910     578
0035025060     172
0021205420      58
0021206930       2
Name: count, dtype: int64

In [19]:
alt = {
"0035025060": 3307.04,
"0021195120": 1424.37,
"0021195190": 2188.67,
"0024015110": 2628.25,
"0021235030": 287.5,
"0021205700": 2669.15,
"0023065180": 847.78,
"2306500106": 1735.91,
"0021206280": 2693.03,
"0035035130": 3834.62,
"0021205670": 1906.25,
"0023065120": 2010.78,
"0023035030": 166.06,
"0021205517": 2556.46,
"0021205513": 3103.83,
"2306500204": 2860.7,
"0021205526": 2093.55,
"0035020300": 2291.55,
"0021237060": 464.94,
"0021205940": 2592.57,
"0021205420": 2541.71,
"0035060210": 2143.87,
"2120500050": 3218.72,
"0021206950": 3259.12,
"0024015508": 2405.41,
"0021205525": 2826.3,
"0021195170": 3049.71,
"2120500135": 2547.75,
"0021205518": 3271.69,
"0023125170": 2805.32,
"0021205910": 2559.04,
"0021205521": 2988.58,
"0035025080": 3184.69,
"0021205515": 2544.25,
"0021205519": 2838.14,
"0021206930": 3222.17,
"2401500040": 2813.47,
"0021205524": 3050.29,
"3505500061": 529.22,
"0023065110": 1337.98,
"0024015380": 2956.61,
"0021205660": 765.72,
}

In [20]:
wsdata["longitud"] = wsdata["codigo_estacion"].map(dict(zip(coor["codigo_estacion"], coor["longitud"])))
wsdata["latitud"] = wsdata["codigo_estacion"].map(dict(zip(coor["codigo_estacion"], coor["latitud"])))
wsdata["altura"] = wsdata["codigo_estacion"].map(alt)

In [21]:
wsdata.drop_duplicates(inplace=True)
wsdata.reset_index(drop=True, inplace=True)

In [22]:
wsdata.head()

,codigo_estacion,fecha_observacion,presion,velocidad,direccion,temperatura,longitud,latitud,altura
0,0021195120,2025-12-15 00:00:00,856.3,2.1,11.0,20.3,-74.393,4.393,1424.37
1,0021195120,2025-12-15 01:00:00,856.8,2.4,27.0,19.5,-74.393,4.393,1424.37
2,0021195120,2025-12-15 02:00:00,857.5,2.9,30.0,18.8,-74.393,4.393,1424.37
3,0021195120,2025-12-15 03:00:00,858.0,2.2,24.0,18.4,-74.393,4.393,1424.37
4,0021195120,2025-12-15 04:00:00,858.1,2.5,27.0,18.1,-74.393,4.393,1424.37


In [23]:
wsdata["fecha_observacion"].sort_values().diff().value_counts()

fecha_observacion
0 days 00:00:00    23986
0 days 00:10:00     2447
Name: count, dtype: int64

In [24]:
fecha_inicio = wsdata["fecha_observacion"].min()
fecha_fin = wsdata["fecha_observacion"].max()
print(f"{fecha_inicio=} , {fecha_fin=}")
rango_tiempo = pd.date_range(fecha_inicio, fecha_fin, freq="10min").to_list()
estaciones_registradas = wsdata["codigo_estacion"].unique()
estaciones_registradas


fecha_inicio=Timestamp('2025-12-15 00:00:00') , fecha_fin=Timestamp('2025-12-31 23:50:00')


array(['0021195120', '0021195170', '0021195190', '0021205420',
       '0021205513', '0021205515', '0021205517', '0021205518',
       '0021205519', '0021205521', '0021205524', '0021205525',
       '0021205526', '0021205660', '0021205670', '0021205700',
       '0021205910', '0021205940', '0021206280', '0021206930',
       '0021206950', '0021235030', '0021237060', '0023035030',
       '0023065110', '0023065120', '0023065180', '0023125170',
       '0024015110', '0024015380', '0024015508', '0035020300',
       '0035025060', '0035025080', '0035035130', '0035060210',
       '2120500050', '2120500135', '2306500106', '2306500204',
       '2401500040', '3505500061'], dtype=object)

In [25]:
estaciones_registradas = wsdata["codigo_estacion"].unique()
estaciones = list()
latitudes = list()
longitudes = list()
alturas = list()

for estacion in estaciones_registradas:
    estaciones += [estacion]*len(rango_tiempo)
    latitudes += [wsdata.loc[wsdata["codigo_estacion"] == estacion, "latitud"].unique()[0]]*len(rango_tiempo)
    longitudes += [wsdata.loc[wsdata["codigo_estacion"] == estacion, "longitud"].unique()[0]]*len(rango_tiempo)
    alturas += [wsdata.loc[wsdata["codigo_estacion"] == estacion, "altura"].unique()[0]]*len(rango_tiempo)
    
    ...

fechas = rango_tiempo * len(estaciones_registradas)

In [26]:
base_estacion_fechas = pd.DataFrame({"codigo_estacion":estaciones, "fecha_observacion":fechas, "latitud":latitudes, "longitud":longitudes, "altura":alturas})
wsdata_full = pd.merge(left=base_estacion_fechas, right=wsdata, on=["codigo_estacion", "fecha_observacion", "latitud", "longitud", "altura"], how="left")

In [27]:
wsdata_full.head()

,codigo_estacion,fecha_observacion,latitud,longitud,altura,presion,velocidad,direccion,temperatura
0,0021195120,2025-12-15 00:00:00,4.393,-74.393,1424.37,856.3,2.1,11.0,20.3
1,0021195120,2025-12-15 00:10:00,4.393,-74.393,1424.37,NaN,NaN,NaN,NaN
2,0021195120,2025-12-15 00:20:00,4.393,-74.393,1424.37,NaN,NaN,NaN,NaN
3,0021195120,2025-12-15 00:30:00,4.393,-74.393,1424.37,NaN,NaN,NaN,NaN
4,0021195120,2025-12-15 00:40:00,4.393,-74.393,1424.37,NaN,NaN,NaN,NaN


In [28]:
wsdata_full = wsdata_full.sort_values(["codigo_estacion", "fecha_observacion"]).reset_index(drop=True)

In [29]:
wsdata_full["vel_u"] = wsdata_full.apply(lambda df: df["velocidad"] * np.sin(np.pi*(df["direccion"]/180)), axis=1)
wsdata_full["vel_v"] = wsdata_full.apply(lambda df: df["velocidad"] * np.cos(np.pi*(df["direccion"]/180)), axis=1)

In [30]:
wsdata_full["segundos"] = (wsdata_full["fecha_observacion"] - fecha_inicio) / np.timedelta64(1, 's')

In [31]:
wsdata_full = wsdata_full[[
    'codigo_estacion',
    'latitud',
    'longitud',
    'altura',
    'segundos',
    'presion',
    'vel_u',
    'vel_v',
    'temperatura',
]].copy()

In [32]:
wsdata_full.head()

,codigo_estacion,latitud,longitud,altura,segundos,presion,vel_u,vel_v,temperatura
0,0021195120,4.393,-74.393,1424.37,0.0,856.3,0.400699,2.061417,20.3
1,0021195120,4.393,-74.393,1424.37,600.0,NaN,NaN,NaN,NaN
2,0021195120,4.393,-74.393,1424.37,1200.0,NaN,NaN,NaN,NaN
3,0021195120,4.393,-74.393,1424.37,1800.0,NaN,NaN,NaN,NaN
4,0021195120,4.393,-74.393,1424.37,2400.0,NaN,NaN,NaN,NaN


In [33]:
path_save_wsdata = Path().cwd().parent / "data" / "raw" / "em_cundinamarca_251212_251231.parquet"
path_save_wsdata_full = Path().cwd().parent / "data" / "raw" / "em_cundinamarca_251212_251231_full.parquet"
wsdata.to_parquet(path_save_wsdata, engine="fastparquet", index=False)
wsdata_full.to_parquet(path_save_wsdata_full, engine="fastparquet", index=False)